In [1]:
from utils import *

/hpi/fs00/home/jim.maar/miniconda3/envs/othello-env/lib/python3.11/site-packages/accelerate/utils/imports.py:274: UserWarning: `ACCELERATE_DISABLE_RICH` is deprecated and will be removed in v0.22.0 and deactivated by default. Please use `ACCELERATE_ENABLE_RICH` if you wish to use `rich`.
  warnings.warn(


In [2]:
device

device(type='cuda')

## TODOS
- normal evaluation!
- mean ablation
- and final evaluation 

In [3]:
board_seqs_int.shape

torch.Size([100000, 60])

In [4]:
board = OthelloBoardState()
print(board.state)
board.umpire(19)
print(board.state)
board.umpire(20)
print(board.state)
board.umpire(21)
print(board.state)
print(board.get_valid_moves())

[[ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0. -1.  1.  0.  0.  0.]
 [ 0.  0.  0.  1. -1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]]
[[ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  1.  0.  0.  0.]
 [ 0.  0.  0.  1. -1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]]
[[ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1. -1.  0.  0.  0.]
 [ 0.  0.  0.  1. -1.  0.  0.  0.]
 [ 0.  0.  0.  1. -1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]]
[[ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  1.  1.  0.  0.]
 [ 0.  0.  0.  1.  1.  0.  0.  0.]
 [ 0.  0.  0.  1.

In [5]:
import functools

In [6]:
act_names = [utils.get_act_name("resid_post", layer) for layer in range(8)]
num_games = 200
fake_cache = get_activation(board_seqs_int, act_names, num_games)
resids = [fake_cache[utils.get_act_name("resid_post", layer)].to(device) for layer in range(8)]
mean_resids = [resid.mean(dim=0) for resid in resids]

Getting activations ...


In [7]:
act_names = [utils.get_act_name("attn_out", layer) for layer in range(8)]
num_games = 200
fake_cache = get_activation(board_seqs_int, act_names, num_games)
attn_outs = [fake_cache[utils.get_act_name("attn_out", layer)].to(device) for layer in range(8)]
mean_attn_out = t.stack([attn_out.mean(dim=0) for attn_out in attn_outs], dim=0)

Getting activations ...


In [8]:
mean_resids[0].shape

torch.Size([59, 512])

In [9]:
indeces_soft = {
    0 : list(range(0, 59)),
    1 : list(range(5, 20)),
    2 : list(range(10, 30)),
    3 : list(range(20, 40)),
    4 : list(range(30, 50)),
    5 : list(range(0, 59)),
    6 : list(range(0, 59)),
    7 : list(range(0, 59)),
}

indeces_soft = {
    0 : list(range(0, 59)),
    1 : list(range(0, 30)),
    2 : list(range(10, 40)),
    3 : list(range(20, 50)),
    4 : list(range(30, 59)),
    5 : list(range(40, 59)),
    6 : list(range(0, 59)),
    7 : list(range(0, 59)),
}
indeces_soft = {
    0 : list(range(0, 59)),
    1 : list(range(0, 59)),
    2 : list(range(10, 59)),
    3 : list(range(20, 59)),
    4 : list(range(30, 59)),
    5 : list(range(40, 59)),
    6 : list(range(0, 59)),
    7 : list(range(0, 59)),
}

indeces_hard = {
    0 : list(range(0, 59)),
    1 : list(range(0, 59)),
    2 : [],
    3 : [],
    4 : [],
    5 : [],
    6 : list(range(0, 59)),
    7 : list(range(0, 59)),
}

In [10]:
# maybe I could actually ablate the attn layers in the upper layers ...
indeces_top_left = {
    0 : [],
    1 : [],
    2 : list(range(0, 10)),
    3 : list(range(0, 20)),
    4 : list(range(0, 30)),
    5 : [],
    6 : [],
    7 : [],
}

indeces_bottum_right = {
    0 : [],
    1 : [],
    2 : list(range(10, 20)),
    3 : list(range(20, 30)),
    4 : list(range(30, 59)),
    5 : [],
    6 : [],
    7 : [],
}

"""indeces_bottum_right = {
    0 : [],
    1 : [],
    2 : [],
    3 : [],
    4 : [],
    5 : [],
    6 : [],
    7 : list(range(59)),
}"""

"""indeces_bottum_right = {
    0 : list(range(0, 30)),
    1 : list(range(0, 30)),
    2 : list(range(0, 30)),
    3 : list(range(0, 30)),
    4 : list(range(0, 30)),
    5 : list(range(0, 30)),
    6 : list(range(0, 30)),
    7 : list(range(0, 30)),
}"""

'indeces_bottum_right = {\n    0 : list(range(0, 30)),\n    1 : list(range(0, 30)),\n    2 : list(range(0, 30)),\n    3 : list(range(0, 30)),\n    4 : list(range(0, 30)),\n    5 : list(range(0, 30)),\n    6 : list(range(0, 30)),\n    7 : list(range(0, 30)),\n}'

In [11]:
indeces_bottum_right = {
    0 : [],
    1 : [],
    2 : [],
    3 : [],
    4 : [],
    5 : [],
    6 : [],
    7 : list(range(0, 59)),
}

In [12]:
indeces_top_left = {
    0 : [],
    1 : [],
    2 : list(range(0, 10)),
    3 : list(range(10, 20)),
    4 : list(range(20, 30)),
    5 : [],
    6 : [],
    7 : [],
}

In [13]:
indeces = indeces_top_left
using_mlp = True # False
using_attn = True # True

def ablation_hook_attn_out(
    attn_out: Float[Tensor, "batch seq d_model"],
    hook: HookPoint,
    pos : int,
) -> Float[Tensor, "batch seq n_heads d_head"]:
    attn_out[:, pos] = t.zeros_like(attn_out[:, pos])
    return attn_out

def ablation_hook_mlp_out(
    mlp_out: Float[Tensor, "batch seq d_model"],
    hook: HookPoint,
    pos : int,
) -> Float[Tensor, "batch seq n_heads d_head"]:
    mlp_out[:, pos] = t.zeros_like(mlp_out[:, pos])
    return mlp_out

def ablation_hook_resid_mid(
    resid_mid: Float[Tensor, "batch seq d_model"],
    hook: HookPoint,
    pos : int,
    layer : int,
    layer0_resid_post: Float[Tensor, "batch seq d_model"],
    attn_out: Float[Tensor, "batch seq d_model"],
    mlp_out_all: Float[Tensor, "batch seq d_model"],
) -> Float[Tensor, "batch seq d_model"]:
    resid_mid[:, pos] = layer0_resid_post[:, pos] + attn_out[:, pos] # + mlp_out_all[:, pos] - mean_attn_out[:layer].sum(dim=0)[pos] # -mean_resids[0][pos] + mean_resids[layer-1][pos]
    return resid_mid

hook_functions = []
for layer in range(0, 7):
    name_attn = utils.get_act_name("attn_out", layer)
    name_mlp = utils.get_act_name("mlp_out", layer)
    for pos in range(0, 59):
        if pos not in indeces[layer]:
            continue
        hook_function_attn = functools.partial(ablation_hook_attn_out, pos=pos)
        hook_function_mlp = functools.partial(ablation_hook_mlp_out, pos=pos)
        if using_attn:
            hook_functions.append((name_attn, hook_function_attn))
        if using_mlp:
            hook_functions.append((name_mlp, hook_function_mlp))

In [14]:
def get_f1_score(legal_moves, logits):
    # overkill tactic would be using k-means to find the best threshold
    preds: Float[Tensor, "batch pos pred"] = logits > 4.0
    batch_size = preds.shape[0]
    tp = 0
    fp = 0
    fn = 0
    for batch in range(batch_size):
        for pos in range(59):
            # indeces where preds[batch, pos] is true
            preds_batch_pos = preds[batch, pos].to(t.int) # .nonzero().squeeue()
            legal_moves_batch_pos = t.zeros(61).to(t.int).to(device)
            legal_moves_batch_pos[legal_moves[batch][pos]] = 1
            tp += (preds_batch_pos * legal_moves_batch_pos).sum()
            fp += (preds_batch_pos * (1 - legal_moves_batch_pos)).sum()
            fn += ((1 - preds_batch_pos) * legal_moves_batch_pos).sum()
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 = 2 * precision * recall / (precision + recall)
    return f1


In [15]:
test_t = t.Tensor([-1, -1, 1, 1, -1])
(test_t > 0).nonzero().squeeze()

tensor([2, 3])

In [16]:
legal_moves_batch_pos = t.zeros(60)
legal_moves = t.Tensor([1, 2, 3]).tolist()
print(legal_moves)
legal_moves_batch_pos[legal_moves] = 1
legal_moves_batch_pos

[1.0, 2.0, 3.0]


tensor([0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.])

In [21]:
use_top_left_hooks = False
use_ablation_hook_resid_mid = True

batch_size = 64
num_games = 1024

all_correct_count = 0
all_correct_count_ablated = 0
all_count = 0

# hook_functions = []

# Ich will gleich into meta shit gehen?
for batch in tqdm(range(0, num_games, batch_size)):
    indeces = t.arange(batch, batch + batch_size)
    games = board_seqs_int[indeces].to(device)

    legal_moves = []
    for i in range(batch, batch+batch_size):
        moves = board_seqs_int[i]
        board = OthelloBoardState()
        legal_moves_board = []
        for move in moves:
            move = to_string(move.item())
            board.umpire(move)
            legal_moves_board_position = board.get_valid_moves()
            legal_moves_board += [legal_moves_board_position]
        legal_moves += [to_int(legal_moves_board)]

    logits = model(games[:, :-1])
    logits, cache = model.run_with_cache(games[:, :-1])
    preds = logits.argmax(dim=-1).to("cpu")

    layer0_resid_post = cache["resid_post", 0]
    new_hooks = []
    mlp_out_all = t.zeros(batch_size, 59, 512).to(device)
    for layer in range(8):
        attn_out = cache["attn_out", layer]
        mlp_out = cache["mlp_out", layer]
        for pos in range(59):
            if pos not in indeces_bottum_right[layer]:
                continue
            temp_hook_fn = functools.partial(ablation_hook_resid_mid, pos=pos, layer=layer, layer0_resid_post=layer0_resid_post, attn_out=attn_out, mlp_out_all=mlp_out_all)
            new_hooks.append((utils.get_act_name("resid_mid", layer), temp_hook_fn))
        mlp_out_all += mlp_out
    
    hooks = []
    if use_top_left_hooks:
        hooks += hook_functions
    if use_ablation_hook_resid_mid:
        hooks += new_hooks

    model.reset_hooks()
    cache_ablated = model.add_caching_hooks()
    logits_ablated = model.run_with_hooks(games[:, :-1], fwd_hooks=hooks)
    cache_ablated = ActivationCache(cache_ablated, model)
    preds_ablated = logits_ablated.argmax(dim=-1).to("cpu")

    batch_count = batch_size * 59
    batch_correct_count = 0
    batch_correct_count_ablated = 0
    for game in range(0, batch_size):
        for pos in range(0, 59):
            if preds[game, pos].item() in legal_moves[game][pos]:
                batch_correct_count += 1
            if preds_ablated[game, pos].item() in legal_moves[game][pos]:
                batch_correct_count_ablated += 1
    all_correct_count += batch_correct_count
    all_correct_count_ablated += batch_correct_count_ablated
    all_count += batch_count

    f1 = get_f1_score(legal_moves, logits)
    f1_ablated = get_f1_score(legal_moves, logits_ablated)

100%|██████████| 16/16 [01:08<00:00,  4.31s/it]


In [22]:
print(all_correct_count / all_count) # 99.8
print(all_correct_count_ablated / all_count) # 72 -> 90 Das ist schon nicht so gut wie ich dachte (Mean ablation might be better) Or reconstucting the same boardstate in the new layer using the probes ... # Visualize the things ... 
print(f1)
print(f1_ablated)

0.9993544756355932
0.9938592425847458
tensor(0.9928, device='cuda:0')
tensor(0.9610, device='cuda:0')


In [ ]:
# Bei Layer 7: 0.99
# Bei Layer 6: 0.61
# Bei Layer 5: 0.87
# Bei Layer 4: 0.93

# layer 7 attn + resid0_post
# 0.9989406779661016
# 0.9941737288135594
# tensor(0.9927, device='cuda:0')
# tensor(0.9659, device='cuda:0')

# layer7 attn
# 0.9989406779661016
# 0.9748411016949152
# tensor(0.9927, device='cuda:0')
# tensor(0.9688, device='cuda:0')

# layer0 resid post
# 0.9989406779661016
# 0.6308262711864406
# tensor(0.9927, device='cuda:0')
# tensor(0.3792, device='cuda:0')

In [157]:
# game = 0
vis_args = VisualzeBoardArguments()
pos = 21
vis_args.start_pos = pos-0
vis_args.end_pos = pos+1
vis_args.layer_start = 0
vis_args.layers = 7
# vis_args.static_image = True
vis_args.include_resid_post = False
vis_args.include_layer_norm = True
vis_args.horizontal_spacing = 0.01
# vis_args.static_image = True

visualize_game(board_seqs_int[0], vis_args, None, cache_ablated)
visualize_game(board_seqs_int[0], vis_args, None, cache)

torch.Size([7, 2, 59, 8, 8])


torch.Size([7, 2, 59, 8, 8])


## Direct Logit Attribution
- Let's take game 0 position 9 where B3 is a legal move and see how the ablated model computes this ...
- First wee see the ablated model does give lower logits (7 rather then 10) and for the non-legal B2 it gives (2.6 rather then -0.1) So it's more indecicive

In [17]:
label = "B6"
tile_int = to_int(label)
pos = 10

In [18]:
print("Score for B3:", logits_ablated[0, pos, tile_int], logits[0, pos, tile_int])
print("Score for B2:", logits_ablated[0, pos, 11], logits[0, pos, 11])

Score for B3: tensor(9.4986, device='cuda:0', grad_fn=<SelectBackward0>) tensor(8.5900, device='cuda:0', grad_fn=<SelectBackward0>)
Score for B2: tensor(0.9015, device='cuda:0', grad_fn=<SelectBackward0>) tensor(-1.2559, device='cuda:0', grad_fn=<SelectBackward0>)


In [19]:
tile_legal_dir = model.W_U[:, tile_int]
tile_legal_dir.shape

torch.Size([512])

In [20]:
neuron = 644
print(cache_ablated["mlp_post", 7][0, pos, neuron])
print(model.W_out[7, neuron] @ tile_legal_dir)
print(model.W_out[7, neuron] @ direction)

tensor(-0.0001, device='cuda:0')
tensor(0.2692, device='cuda:0', grad_fn=<DotBackward0>)


NameError: name 'direction' is not defined

In [52]:
resid_mid = cache_ablated["resid_mid", 3]
resid_post_0 = cache_ablated["resid_post", 0]
attn_out = cache_ablated["attn_out", 3]
resid_mid[0, pos, :10], resid_post_0[0, pos, :10] + attn_out[0, pos, :10]

(tensor([ 0.2749,  0.2674, -0.4987, -0.1209,  0.7342,  0.2653, -0.3397,  0.0605,
          0.2941,  0.1413], device='cuda:0'),
 tensor([ 0.2573,  0.5321, -0.4156, -0.1945,  0.8401,  0.2988, -0.2451,  0.3665,
          0.2834,  0.0162], device='cuda:0'))

In [53]:
resid_0_post = cache_ablated["resid_post", 0]
resid_0_post_clean = cache["resid_post", 0]
lay = 7
attn_out_7 = cache["attn_out", lay]
mlp_out_7 = cache_ablated["mlp_out", lay]
resid_post_7 = cache_ablated["resid_post", lay]
# cache_ablated["resid_post", 2][0, pos][:10], cache["resid_post", 2][0, pos][:10]
(resid_0_post[0, pos] + attn_out_7[0, pos] + mlp_out_7[0, pos])[:10], resid_post_7[0, pos][:10]

(tensor([ 0.3842,  0.1986,  0.4793,  0.0716,  0.0481,  0.5681, -0.7169,  0.0637,
         -0.3523,  0.2360], device='cuda:0'),
 tensor([ 0.3842,  0.1986,  0.4793,  0.0716,  0.0481,  0.5681, -0.7169,  0.0637,
         -0.3523,  0.2360], device='cuda:0'))

In [54]:
resid_post = cache_ablated["resid_post", 7] # Theres still a diff between resid post and the output logits
print(resid_post[0, pos+0] @ tile_legal_dir)

tensor(5.3270, device='cuda:0', grad_fn=<DotBackward0>)


In [55]:
def direciton_attribution(game_idx, layer, direction):
    attributions = {}
    for module_name2 in ["resid_pre", "attn_out", "mlp_out", "resid_post"]:
        if module_name2 == "resid_pre" and layer != 0:
            resid : Float[Tensor, "pos d_model"] = cache_ablated[module_name2, 1][game_idx]
        elif module_name2 == "attn_out":
            resid : Float[Tensor, "pos d_model"] = cache[module_name2, layer][game_idx]
        else:
            resid : Float[Tensor, "pos d_model"] = cache_ablated[module_name2, layer][game_idx]
        attribution = einops.einsum(resid, direction, "pos d_model, d_model -> pos")
        attributions[f"{module_name2} ({layer})"] = attribution[:].unsqueeze(0)
    for head in range(8):
        # print(focus_cache.keys())
        resid : Float[Tensor, "pos d_model"] = cache_ablated["z", layer][game_idx, :, head, :]
        resid = einops.einsum(resid, model.W_O[layer, head], "pos d_head, d_head d_model -> pos d_model").detach()
        # print(resid.shape)
        attribution = einops.einsum(resid, direction, "pos d_model, d_model -> pos")
        attributions[f"head_{head} ({layer})"] = attribution[:].unsqueeze(0)
    return attributions

def plot_direction_attribution(game_idx, layer, direction, title_text, save=False):
    attributions = direciton_attribution(game_idx, layer, direction)
    fig = make_subplots(rows=len(attributions), cols=1, subplot_titles=list(attributions.keys()), shared_xaxes=True)
    abs_max = max([t.abs(attributions[module_name][:, 1:]).max().item() for module_name in attributions])
    for i, module_name in enumerate(attributions):
        # print(attributions[module_name].shape)
        fig.add_trace(go.Heatmap(z=attributions[module_name].cpu().numpy(), colorscale="RdBu", zmin=-abs_max, zmax=abs_max), row=i+1, col=1)
        fig.update_yaxes(showticklabels=False, row=i+1, col=1)
    fig.update_layout(title_text=title_text, height = 1000, width = 1500)
    if save:
        save_plotly(fig, title_text)
    fig.show()

def plot_neuron_attribution(game_idx, layer, position, direction : Float[Tensor, "d_model"], title_text, cache=cache_ablated):
    W_out : Float[Tensor, "d_mlp d_model"] = model.W_out[layer].detach()
    activations = cache["post", layer][game_idx]
    attributions = activations[position] * einops.einsum(W_out, direction, "d_mlp d_model, d_model -> d_mlp")
    print(attributions.shape)
    # Make a plotly scatter plot. y axis is the attribution, x axis is the neuron index
    fig = px.scatter(x=list(range(attributions.shape[0])), y=attributions.cpu().numpy().flatten(), labels={"x": "Neuron Index", "y": "Attribution"})
    fig.update_layout(title_text=title_text)
    fig.update_yaxes(exponentformat = 'E')
    fig.show()

tile_label = label
direction_label = "legal"
game = 0
layer = 7
tile = label_to_tuple(tile_label)
direction = get_probe(layer, "legal", "post")[0, :, *tile, get_direction_int(direction_label)]
direction = tile_legal_dir.detach()

plot_direction_attribution(game, layer, direction, title_text=f"Attributions of Layer {layer} to the {direction_label} direction of {tile_label} (We are looking at Position 12) Game {game}", save=False)

position = pos
print(pos)

# direction_label = "legal"
# direction = get_probe(layer, "legal", "post")[0, :, *tile, get_direction_int(direction_label)]
plot_neuron_attribution(game, layer, position, direction, title_text=f"Attributions of Layer {layer} Neurons to the {direction_label} direction of {tile_label} in game 0 (Posiiton {position})")

10
torch.Size([2048])


In [81]:
# Wenn ich die obere linke hälfte ablate.
#     Nur die Attn layer: 0.990
#     MLP und Attn layer: 0.90

# Also haben die späteren layer nicht unbedingt die capacity zum flippen

# Top Accuracy Embedding: 30%
# Top Accuracy Layer 0: 79%
# Top Accuracy Layer 1: 87%

## Old

In [ ]:
'''def head_ablation_hook(
    v: Float[Tensor, "batch seq n_heads d_head"],
    hook: HookPoint,
    head_index_to_ablate: int,
) -> Float[Tensor, "batch seq n_heads d_head"]:
    v[:, :, head_index_to_ablate, :] = 0.0
    return v

def cross_entropy_loss(logits, tokens):
    '''
    # Computes the mean cross entropy between logits (the model's prediction) and tokens (the true values).
    '''
    log_probs = F.log_softmax(logits, dim=-1)
    pred_log_probs = t.gather(log_probs[:, :-1], -1, tokens[:, 1:, None])[..., 0]
    return -pred_log_probs.mean()

def get_ablation_scores(
    model: HookedTransformer,
    tokens: Int[Tensor, "batch seq"],
) -> Float[Tensor, "n_layers n_heads"]:
    """
    Returns a tensor of shape (n_layers, n_heads) containing the increase in cross entropy loss from ablating the output of each head.
    """
    # Initialize an object to store the ablation scores
    ablation_scores = t.zeros((model.cfg.n_layers, model.cfg.n_heads), device=model.cfg.device)

    # Calculating loss without any ablation, to act as a baseline
    model.reset_hooks()
    seq_len = (tokens.shape[1] - 1) // 2
    logits = model(tokens, return_type="logits")
    loss_no_ablation = cross_entropy_loss(logits[:, -seq_len:], tokens[:, -seq_len:])

    for layer in tqdm(range(model.cfg.n_layers)):
        for head in range(model.cfg.n_heads):
            # Use functools.partial to create a temporary hook function with the head number fixed
            temp_hook_fn = functools.partial(head_ablation_hook, head_index_to_ablate=head)
            # Run the model with the ablation hook
            ablated_logits = model.run_with_hooks(
                tokens, fwd_hooks=[(utils.get_act_name("v", layer), temp_hook_fn)]
            )
            # Calculate the loss difference
            loss = cross_entropy_loss(ablated_logits[:, -seq_len:], tokens[:, -seq_len:])
            # Store the result, subtracting the clean loss so that a value of zero means no change in loss
            ablation_scores[layer, head] = loss - loss_no_ablation

    return ablation_scores


ablation_scores = get_ablation_scores(model, rep_tokens)
tests.test_get_ablation_scores(ablation_scores, model, rep_tokens)'''